# TUT11-1 Graph Processing
## **Graph representation**

### **Graph Structure**
Mathematically, a graph $\mathcal{G}$ is defined as a tuple of a set of nodes/vertices $V$, and a set of edges/links $E$: $\mathcal{G}=(V,E)$. Each edge is a pair of two vertices, and represents a connection between them as shown in the figure.

<center width="100%" style="padding:10px"><img src="https://github.com/phlippe/uvadlc_notebooks/blob/master/docs/tutorial_notebooks/tutorial7/example_graph.svg?raw=1" width="250px"></center>

The vertices are $V=\{1,2,3,4\}$, and edges $E=\{(1,2), (2,3), (2,4), (3,4)\}$. Note that for simplicity, we assume the graph to be undirected and hence don't add mirrored pairs like $(2,1)$. 
In application, vertices and edge can often have specific attributes, and edges can even be directed. 

### **Adjacency Matrix**
The **adjacency matrix** $A$ is a square matrix whose elements indicate whether pairs of vertices are adjacent, i.e. connected, or not. 
In the simplest case, $A_{ij}$ is 1 if there is a connection from node $i$ to $j$, and otherwise 0. If we have edge attributes or different categories of edges in a graph, this information can be added to the matrix as well. For an undirected graph, keep in mind that $A$ is a symmetric matrix ($A_{ij}=A_{ji}$). For the example graph above, we have the following adjacency matrix:

$$
A = \begin{bmatrix}
    0 & 1 & 0 & 0\\
    1 & 0 & 1 & 1\\
    0 & 1 & 0 & 1\\
    0 & 1 & 1 & 0
\end{bmatrix}
$$

Alternatively, we could also define a sparse adjacency matrix with which we can work as if it was a dense matrix, but allows more memory-efficient operations. PyTorch supports this with the sub-package `torch.sparse` ([documentation](https://pytorch.org/docs/stable/sparse.html)).

## 1. Import libraries

In [1]:
import numpy as np
import scipy.sparse as sp
import torch

## 2. Load features

In [2]:
path = '../input/aist4010-spring2022-a3/data/'

idx_features = np.loadtxt(path + "features.txt", dtype=np.dtype(str)) 
features = idx_features[:, 1:]

In [3]:
idx_features, features

In [4]:
idx_features.shape, features.shape

In [5]:
# Compressed Sparse Row matrix
features = sp.csr_matrix(features, dtype=np.float32)  # features   2707 * 1433  

## 3. Load Labels

### 1) Load train and val data

In [6]:
train_data = np.loadtxt(path + "train_labels.csv", delimiter=",", dtype=np.dtype(str))
train_idx, train_labels = train_data[1:, 0], train_data[1:, 1]

val_data = np.loadtxt(path + "val_labels.csv", delimiter=",", dtype=np.dtype(str))
val_idx, val_labels = val_data[1:, 0], val_data[1:, 1] # one-hot encoding labels 2708 * 7

In [7]:
train_idx[:10], train_labels[:10]

### 2) Load test idx

In [8]:
test_idx, _ = np.loadtxt(path + "test_idx.csv", delimiter=",", dtype=np.dtype(str), unpack = True)
test_idx = test_idx[1:]

all_idx = np.concatenate((train_idx, val_idx, test_idx), axis = 0)

In [9]:
test_idx.shape, all_idx.shape

### 3) One-hot encoding

In [10]:
def encode_onehot(labels):
    classes = set(labels)

    class_dict = {c:i for i, c in enumerate(classes)}
    classes_onehot_dict = {c: np.identity(len(classes))[i, :] for i, c in
                    enumerate(classes)}
    labels_onehot = np.array(list(map(classes_onehot_dict.get, labels)),
                             dtype=np.int32)
    return labels_onehot, class_dict

train_labels, class_dict = encode_onehot(train_labels)
val_labels, _ = encode_onehot(val_labels)

In [11]:
class_dict

## 4. Build graph
### 1) Load nodes

In [12]:
idx = np.array(idx_features[:, 0], dtype=np.int32)          # nodes names               2707
idx_map = {j: i for i, j in enumerate(idx)}                 # nodes mapping    'names' : 'idx'

In [13]:
dict(list(idx_map.items())[:10])

### 2) Load edges

In [14]:
edges_unordered = np.genfromtxt(path + "edges.txt", dtype=np.int32)       # node1, node2                        
edges = np.array(list(map(idx_map.get, edges_unordered.flatten())),       
                 dtype=np.int32).reshape(edges_unordered.shape)           # node_idx1, node_idx2      5427 * 2

In [15]:
edges.shape, edges[:10]

### 3) Build adjacency matrix

In [16]:
# build graph
# A sparse matrix in COOrdinate format.
adj = sp.coo_matrix((np.ones(edges.shape[0]), (edges[:, 0], edges[:, 1])),
                    shape=(all_idx.shape[0], all_idx.shape[0]),
                    dtype=np.float32)                                     # adjacency matrix          2707 * 2707

# build symmetric adjacency matrix
adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)       # symmetric adjacency matrix

In [17]:
adj

### 4) Normalize

In [18]:
def normalize(mx):
    """Row-normalize sparse matrix"""
    rowsum = np.array(mx.sum(1))
    r_inv = np.power(rowsum, -1).flatten()
    r_inv[np.isinf(r_inv)] = 0.
    r_mat_inv = sp.diags(r_inv)
    mx = r_mat_inv.dot(mx)
    return mx

# normalize
features_n = normalize(features)
adj_n = normalize(adj + sp.eye(adj.shape[0]))